In [148]:
import itertools
import json
from pathlib import Path

import pandas as pd

In [149]:
def sorted_tuple(x):
    return tuple(sorted(x))

In [150]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/drop-mini-manual").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
next(iter(experiments), None)

28 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'dataset': {'path': 'bdsaglam/drop-musique-mini',
   'name': 'default',
   'split': 'train'},
  'qa': {'model': 'llama-3-8b',
   'temperature': 0.0,
   'system_prompt': 'direct/format-few.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'manual/direct.json',
   'n_shot': 0,
   'n_sc': 1},
  'run': 1},
 'metrics': {'exact_match': 0.37,
  'f1': 0.5993370884585718,
  'fuzzy_match': 0.41,
  '0hops': {'exact_match': 0.37,
   'f1': 0.5993370884585718,
   'fuzzy_match': 0.41},
  'gen_token_count': {'all': {'count': 100.0,
    'mean': 6.41,
    'std': 4.431510219810265,
    'min': 3.0,
    '25%': 4.0,
    '50%': 5.0,
    '75%': 7.0,
    'max': 28.0},
   'success': {'count': 65.0,
    'mean': 5.3076923076923075,
    'std': 2.8718628368789054,
    'min': 3.0,
    '25%': 4.0,
    '50%': 5.0,
    '75%': 5.0,
    'max': 22.0},
   'fail': {'count': 35.0,
    'mean': 8.457142857142857,
    'std': 5.91778405040492

In [151]:
df = pd.json_normalize(experiments)
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)
print(f"{len(df)} experiments before preprocessing")
df.head()

28 experiments before preprocessing


,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.system_prompt,params.qa.user_prompt_template,...,metrics.gen_token_count.success.75%,metrics.gen_token_count.success.max,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max
0,workspace,workspace,None,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.0,direct/format-few.txt,cq.txt,...,5.0,22.0,35.0,8.457143,5.917784,3.0,5.0,6.0,9.0,28.0
1,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,4c3a8040449c9603bec5032b54162c41f53c442b,bifid-snow,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,430.5,1068.0,65.0,799.169231,1610.523271,159.0,269.0,342.0,440.0,8193.0
2,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,a5354126b8078a8578850a29c2b8f2a34a1ac2d3,ovoid-puds,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,359.0,1187.0,63.0,521.571429,1005.241196,159.0,259.0,344.0,448.0,8065.0
3,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,80698cde292df8ac95412aa9953826f94f6bba76,piano-mean,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,355.5,524.0,73.0,380.246575,212.387276,123.0,255.0,324.0,438.0,1308.0
4,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,177129695d4899f693b218a96e449e6a2dbf2675,spumy-sign,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,354.5,691.0,69.0,711.971014,1534.491360,159.0,255.0,321.0,444.0,8193.0


In [152]:
param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]

In [153]:
df.dropna(subset=metric_cols[:2], inplace=True, how="any")
df.drop_duplicates(subset=param_cols, keep='last', inplace=True)

print(f"{len(df)} experiments after preprocessing")
df.head()

26 experiments after preprocessing


,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.system_prompt,params.qa.user_prompt_template,...,metrics.gen_token_count.success.75%,metrics.gen_token_count.success.max,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max
1,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,4c3a8040449c9603bec5032b54162c41f53c442b,bifid-snow,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,430.5,1068.0,65.0,799.169231,1610.523271,159.0,269.0,342.0,440.0,8193.0
2,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,a5354126b8078a8578850a29c2b8f2a34a1ac2d3,ovoid-puds,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,359.0,1187.0,63.0,521.571429,1005.241196,159.0,259.0,344.0,448.0,8065.0
3,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,80698cde292df8ac95412aa9953826f94f6bba76,piano-mean,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,355.5,524.0,73.0,380.246575,212.387276,123.0,255.0,324.0,438.0,1308.0
4,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,177129695d4899f693b218a96e449e6a2dbf2675,spumy-sign,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,354.5,691.0,69.0,711.971014,1534.491360,159.0,255.0,321.0,444.0,8193.0
5,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,50f737766dbdddf92ac692d95a8aff45e9468c32,forte-haet,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,369.5,932.0,73.0,437.369863,814.629679,123.0,227.0,317.0,436.0,7161.0


In [154]:
def parse_qa_technique(row):
    model = row['params.qa.model']
    if 'r1' in model:
        base_technique = 'REASONING'
    else:
        base_technique = row["params.qa.system_prompt"].split("/")[0].replace(".txt", "").upper()
    sc = int(row["params.qa.n_sc"])
    sc_suffix = f"-SC({sc})" if sc > 1 else ""
    return f"{base_technique}{sc_suffix}"

df["params.qa.technique"] = df.apply(parse_qa_technique, axis=1)

In [155]:
def remove_ext(filename):
    return filename.split(".", 1)[0]

def parse_instruction(row):
    sp = remove_ext(row["params.qa.system_prompt"]).lower()
    up = remove_ext(row["params.qa.user_prompt_template"]).lower().replace(".txt", "")
    fs = remove_ext(row["params.qa.few_shot_examples"]).lower()
    return f"{sp}:{up}:{fs}"

df["params.qa.instruction"] = df.apply(parse_instruction, axis=1)

In [156]:
# make temperature 1 decimal
df["params.qa.temperature"] = df["params.qa.temperature"].round(1)

In [157]:
for col in param_cols:
    values = list(df[col].unique())
    print(f"- {col}: {values}")
    print()

- params.dataset.path: ['bdsaglam/drop-musique-mini']

- params.dataset.name: ['default']

- params.dataset.split: ['train']

- params.qa.model: ['deepseek-r1-llama-8b', 'llama-3-8b', 'qwen-2.5-32b']

- params.qa.temperature: [0.0, 0.5]

- params.qa.system_prompt: ['empty.txt', 'direct/format-few.txt', 'ccot/format-thought.txt']

- params.qa.user_prompt_template: ['icq-format.txt', 'cq.txt']

- params.qa.few_shot_examples: ['manual/reasoning.json', 'manual/direct.json', 'manual/ccot.json']

- params.qa.n_shot: [0, 1]

- params.qa.n_sc: [1]

- params.run: [3, 1, 2]



In [158]:
df.to_json('exps.jsonl', orient='records', lines=True)

## Setup remaining experiments

In [159]:
def produce_experiment_configs(common_params, varying_params):
    # Generate all possible combinations of parameters
    varying_params = {**common_params, **varying_params}
    keys = varying_params.keys()
    values = varying_params.values()
    for instance in itertools.product(*values):
        yield dict(zip(keys, instance))

In [160]:
def produce_all_experiment_configs(common_params: dict, varying_params_list: list[dict]):
    for params in varying_params_list:
        for exp_config in produce_experiment_configs(common_params, params):
            yield exp_config

In [161]:
common_params = {
    "params.dataset.path": ["bdsaglam/drop-musique-mini"],
    "params.dataset.split": ["train"],
    "params.qa.temperature": [
        0.0,
        0.5,
    ],
    "params.qa.user_prompt_template": ["cq.txt"],
    "params.qa.n_sc": [
        1,
    ],
    "params.qa.n_shot": [
        0,
        1,
    ],
    "params.run": [
        1,
        2,
        3,
    ],
}

In [162]:
varying_params_list = [
    {
        "params.qa.model": [
            "llama-3-8b",
            # "llama-3.3-70b",
        ],
        "params.qa.system_prompt": ["direct/format-few.txt"],
        "params.qa.few_shot_examples": ["manual/direct.json"],
    },
    {
        "params.qa.model": [
            "llama-3-8b",
            # "llama-3.3-70b",
        ],
        "params.qa.system_prompt": ["cot/format-thought.txt"],
        "params.qa.few_shot_examples": ["manual/cot.json"],
    },
    {
        "params.qa.model": [
            "llama-3-8b",
            # "llama-3.3-70b",
        ],
        "params.qa.system_prompt": ["ccot/format-thought.txt"],
        "params.qa.few_shot_examples": ["manual/ccot.json"],
    },
    {
        "params.qa.model": [
            "deepseek-r1-llama-8b",
            # "deepseek-r1-llama-70b",
        ],
        "params.qa.system_prompt": ["empty.txt"],
        "params.qa.user_prompt_template": ["icq-format.txt"],
        "params.qa.few_shot_examples": ["manual/reasoning.json"],
    },
]

In [163]:
exp_configs = list(produce_all_experiment_configs(common_params, varying_params_list))
target_params = list(exp_configs[0].keys())
print(f"{len(exp_configs)} experiment configurations")
print(target_params)

48 experiment configurations
['params.dataset.path', 'params.dataset.split', 'params.qa.temperature', 'params.qa.user_prompt_template', 'params.qa.n_sc', 'params.qa.n_shot', 'params.run', 'params.qa.model', 'params.qa.system_prompt', 'params.qa.few_shot_examples']


In [164]:
if len(df):
    existing_configs = df[target_params].to_dict(orient="records")
    existing_configs[0]
else:
    existing_configs = []

print("Existing exps:", len(existing_configs))

Existing exps: 26


In [165]:
# find the missing configurations
missing_configs = [
    dict(kv)
    for kv in list(
        {tuple(sorted(config.items())) for config in exp_configs}
        - {tuple(sorted(config.items())) for config in existing_configs}
    )
]
print(f"{len(missing_configs)} missing configurations")

23 missing configurations


In [166]:
def make_command(exp_config):
    lines = ["dvc exp run --queue"]
    for target_param in target_params:
        arg_name = target_param.split(".", 1)[-1]
        arg_value = exp_config[target_param]
        lines.append(f"-S {arg_name}='{arg_value}'")

    command = " \\\n    ".join(lines)
    return command

In [167]:
with open("run.sh", "w") as f:
    f.write("#!/bin/sh\n\n")
    for exp_config in missing_configs:
        f.write(make_command(exp_config))
        f.write("\n\n")

## Inspect

In [168]:
df

,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.system_prompt,params.qa.user_prompt_template,...,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max,params.qa.technique,params.qa.instruction
1,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,4c3a8040449c9603bec5032b54162c41f53c442b,bifid-snow,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,65.0,799.169231,1610.523271,159.0,269.00,342.0,440.00,8193.0,REASONING,empty:icq-format:manual/reasoning
2,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,a5354126b8078a8578850a29c2b8f2a34a1ac2d3,ovoid-puds,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,63.0,521.571429,1005.241196,159.0,259.00,344.0,448.00,8065.0,REASONING,empty:icq-format:manual/reasoning
3,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,80698cde292df8ac95412aa9953826f94f6bba76,piano-mean,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,73.0,380.246575,212.387276,123.0,255.00,324.0,438.00,1308.0,REASONING,empty:icq-format:manual/reasoning
4,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,177129695d4899f693b218a96e449e6a2dbf2675,spumy-sign,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,69.0,711.971014,1534.491360,159.0,255.00,321.0,444.00,8193.0,REASONING,empty:icq-format:manual/reasoning
5,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,50f737766dbdddf92ac692d95a8aff45e9468c32,forte-haet,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,73.0,437.369863,814.629679,123.0,227.00,317.0,436.00,7161.0,REASONING,empty:icq-format:manual/reasoning
6,be2593eecf7fd1d290f7efd77bf9f142ee3d7e10,230416cf6e8d2b13ebf73d509ff067baf737ebd3,heady-love,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.0,empty.txt,icq-format.txt,...,73.0,482.643836,946.476321,130.0,233.00,332.0,436.00,7930.0,REASONING,empty:icq-format:manual/reasoning
8,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,e7197b9530afa0b74197ac5e6e711e3ccd1d4b0e,hated-fore,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,59.0,579.779661,1048.349393,169.0,260.50,342.0,439.00,6973.0,REASONING,empty:icq-format:manual/reasoning
9,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,39ef92d375302bce71bfecb1c83a632e0dba789f,whity-wave,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,76.0,387.815789,200.796329,149.0,250.00,336.5,470.75,1166.0,REASONING,empty:icq-format:manual/reasoning
10,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,de60a8a11f645e0b77cf84ffdabb82aacb3b0123,spicy-bael,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,60.0,423.283333,332.267816,168.0,255.50,334.5,460.25,2113.0,REASONING,empty:icq-format:manual/reasoning
11,0dc1ac9f99cc3c539cdd54ae3981e8c4f52e6bd3,402223512ca8b9922f56a87af12c575694e2727f,sport-kale,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-8b,0.5,empty.txt,icq-format.txt,...,67.0,381.910448,202.044904,159.0,257.00,317.0,403.50,966.0,REASONING,empty:icq-format:manual/reasoning
